In [ ]:
pip install fastf1

In [ ]:
#Import the required libraries

import fastf1 as f1
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#We gain access to the folder where the txt file with the passwords is located.

from google.colab import drive

drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [ ]:
#We generate a dictionary with the information to connect to Amazon RDS database

Ruta= r"/content/drive/MyDrive"

with open("{}/Capstone_pass.txt".format(Ruta), "r") as file:
    contenido = file.read()

contenido_1= contenido.split(",")

list_1= []

for x in contenido_1:
    element_1= x.split("=")
    list_1.append(element_1)

dict_pass= {}

for y in list_1:
    dict_pass[y[0].strip()]= y[1].strip()

In [ ]:
#Each parameter is stored in one variable

host = dict_pass["host"]
port = dict_pass["port"]
user = dict_pass["user"]
password = dict_pass["password"]
dbname = dict_pass["dbname"]

In [ ]:
#Function to create the tables on the Amazon RDS database

def Create_Tables_Func():

    connection = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=dbname
    )

    cursor = connection.cursor()

    create_table1_query = '''
    CREATE TABLE IF NOT EXISTS "Race_Results" (
        "Year" INT,
        "Event" INT,
        "Session" VARCHAR(10),
        "TeamName" VARCHAR(255),
        "DriverNumber" INT,
        "Abbreviation" VARCHAR(10),
        "FullName" VARCHAR(255),
        "Position" INT,
        "GridPosition" INT,
        "Status" VARCHAR(50),
        "Time" INTERVAL,
        "Time_sec" DECIMAL(9, 4),
        "Points" DECIMAL(5, 2)
    );
    '''

    cursor.execute(create_table1_query)

    create_table2_query = '''
    CREATE TABLE IF NOT EXISTS "Quali_Results" (
        "Year" INT,
        "Event" INT,
        "Session" VARCHAR(10),
        "TeamName" VARCHAR(255),
        "DriverNumber" INT,
        "Abbreviation" VARCHAR(10),
        "FullName" VARCHAR(255),
        "ClassifiedPosition" INT,
        "Q1" INTERVAL,
        "Q1_sec" DECIMAL(9, 4),
        "Q2" INTERVAL,
        "Q2_sec" DECIMAL(9, 4),
        "Q3" INTERVAL,
        "Q3_sec" DECIMAL(9, 4)
    );
    '''

    cursor.execute(create_table2_query)

    create_table3_query = '''
    CREATE TABLE IF NOT EXISTS "Events" (
        "Year" INT,
        "Event" INT,
        "RoundNumber" INT,
        "Location" TEXT,
        "OfficialEventName" TEXT,
        "EventDate" TIMESTAMP,
        "EventName" TEXT,
        "EventFormat" TEXT,
        "Quali_Date" TIMESTAMP WITH TIME ZONE,
        "Quali_DateUtc" TIMESTAMP,
        "Race_Date" TIMESTAMP WITH TIME ZONE,
        "Race_DateUtc" TIMESTAMP
    );
    '''

    cursor.execute(create_table3_query)

    create_table4_query = '''
    CREATE TABLE IF NOT EXISTS "Race_Weather" (
        "Year" INT,
        "Event" INT,
        "Session" VARCHAR(10),
        "Time" INTERVAL,
        "Time_sec" DECIMAL(9, 4),
        "AirTemp" DECIMAL(5, 2),
        "Humidity" DECIMAL(5, 2),
        "Pressure" DECIMAL(7, 2),
        "Rainfall" VARCHAR(10),
        "TrackTemp" DECIMAL(5, 2),
        "WindDirection" INT,
        "WindSpeed" DECIMAL(5, 2)
    );
    '''

    cursor.execute(create_table4_query)

    create_table5_query = '''
    CREATE TABLE IF NOT EXISTS "Quali_Weather" (
        "Year" INT,
        "Event" INT,
        "Session" VARCHAR(10),
        "Time" INTERVAL,
        "Time_sec" DECIMAL(9, 4),
        "AirTemp" DECIMAL(5, 2),
        "Humidity" DECIMAL(5, 2),
        "Pressure" DECIMAL(7, 2),
        "Rainfall" VARCHAR(10),
        "TrackTemp" DECIMAL(5, 2),
        "WindDirection" INT,
        "WindSpeed" DECIMAL(5, 2)
    );
    '''

    cursor.execute(create_table5_query)

    create_table6_query = '''
    CREATE TABLE IF NOT EXISTS "Race_Laps" (
        "Year" INT,
        "Event" INT,
        "Session" VARCHAR(10),
        "Time" INTERVAL,
        "Time_sec" DECIMAL(9, 4),
        "Driver" VARCHAR(10),
        "DriverNumber" INT,
        "LapTime" INTERVAL,
        "LapTime_sec" DECIMAL(9, 4),
        "LapNumber" INT,
        "PitOutTime" INTERVAL,
        "PitOutTime_sec" DECIMAL(9, 4),
        "PitInTime" INTERVAL,
        "PitInTime_sec" DECIMAL(9, 4),
        "Sector1Time" INTERVAL,
        "Sector1Time_sec" DECIMAL(9, 4),
        "Sector2Time" INTERVAL,
        "Sector2Time_sec" DECIMAL(9, 4),
        "Sector3Time" INTERVAL,
        "Sector3Time_sec" DECIMAL(9, 4),
        "Sector1SessionTime" INTERVAL,
        "Sector1SessionTime_sec" DECIMAL(9, 4),
        "Sector2SessionTime" INTERVAL,
        "Sector2SessionTime_sec" DECIMAL(9, 4),
        "Sector3SessionTime" INTERVAL,
        "Sector3SessionTime_sec" DECIMAL(9, 4),
        "SpeedI1" DECIMAL(5, 2),
        "SpeedI2" DECIMAL(5, 2),
        "SpeedFL" DECIMAL(5, 2),
        "SpeedST" DECIMAL(5, 2),
        "Compound" VARCHAR(30),
        "TyreLife" DECIMAL(5, 2),
        "FreshTyre" VARCHAR(10),
        "Team" VARCHAR(255),
        "LapStartTime" INTERVAL,
        "LapStartTime_sec" DECIMAL(9, 4),
        "LapStartDate" TIMESTAMP,
        "TrackStatus" INT,
        "Position" INT
    );
    '''

    cursor.execute(create_table6_query)

    create_table7_query = '''
    CREATE TABLE IF NOT EXISTS "Telemetry_Best_Lap" (
        "Year" INT,
        "Event" INT,
        "Session" VARCHAR(10),
        "DriverNumber" INT,
        "X" DECIMAL(9, 2),
        "Y" DECIMAL(9, 2),
        "Z" DECIMAL(9, 2),
        "Speed" DECIMAL(6, 2),
        "nGear" INT,
        "RPM" INT,
        "Throttle" INT,
        "Brake" VARCHAR(30),
        "Date" TIMESTAMP,
        "Time" INTERVAL,
        "Time_sec" DECIMAL(9, 2),
        "SessionTime" INTERVAL,
        "SessionTime_sec" DECIMAL(9, 2)
    );
    '''

    cursor.execute(create_table7_query)

    create_table8_query = '''
    CREATE TABLE IF NOT EXISTS "Photo_Link" (
        "FullName" TEXT,
        "Photo" TEXT,
        "Country" TEXT,
        "Team" TEXT
    );
    '''

    cursor.execute(create_table8_query)

    create_table9_query = '''
    CREATE TABLE IF NOT EXISTS "Logo_Link" (
        "TeamName" TEXT,
        "Logo" TEXT,
        "Country" TEXT
    );
    '''

    cursor.execute(create_table9_query)

    create_table10_query = '''
    CREATE TABLE IF NOT EXISTS "Champs" (
        "FullName" TEXT,
        "Champ" INT,
        "Wins_2023" INT
    );
    '''

    cursor.execute(create_table10_query)

    create_table11_query = '''
    CREATE TABLE IF NOT EXISTS "Pred" (
        "Column1" INT,
        "FullName" TEXT,
        "TeamName" TEXT,
        "Year" INT,
        "RoundNumber" INT,
        "Position_Q" INT,
        "Q1_Q" NUMERIC(10, 8),
        "Q2_Q" NUMERIC(10, 8),
        "Q3_Q" NUMERIC(10, 8),
        "has_rain_Q" TEXT,
        "has_rain_R" TEXT,
        "Country" TEXT,
        "Location" TEXT,
        "EventFormat" TEXT,
        "Q1_Q_Rank" INT,
        "Q2_Q_Rank" INT,
        "Q3_Q_Rank" INT,
        "DriverPointsBefore" INT,
        "TeamPointsBefore" INT,
        "RawPredictedPosition" INT,
        "CleanPredictedPosition" INT,
        "ActualPosition" INT
    );
    '''

    cursor.execute(create_table11_query)

    create_table12_query = '''
    CREATE TABLE IF NOT EXISTS "Sprint_Results" (
        "Year" INT,
        "Event" INT,
        "Session" VARCHAR(10),
        "TeamName" VARCHAR(255),
        "DriverNumber" INT,
        "Abbreviation" VARCHAR(10),
        "FullName" VARCHAR(255),
        "Position" INT,
        "GridPosition" INT,
        "Status" VARCHAR(50),
        "Time" INTERVAL,
        "Time_sec" DECIMAL(9, 4),
        "Points" DECIMAL(5, 2)
    );
    '''

    cursor.execute(create_table12_query)

    create_table13_query = '''
    CREATE TABLE IF NOT EXISTS "Driver_Info" (
        "DriverNumber" INT,
        "BroadcastName" TEXT,
        "Abbreviation" VARCHAR(10),
        "DriverId" TEXT,
        "TeamName" TEXT,
        "TeamColor" VARCHAR(255),
        "TeamId" TEXT,
        "FirstName" TEXT,
        "LastName" TEXT,
        "FullName" TEXT,
        "HeadshotUrl" TEXT,
        "CountryCode" VARCHAR(10),
        "Position" INT,
        "ClassifiedPosition" VARCHAR(10),
        "GridPosition" INT,
        "Q1" TEXT,
        "Q2" TEXT,
        "Q3" TEXT,
        "Time" TEXT,
        "Status" TEXT,
        "Points" INT,
        "Event" INT
    );
    '''

    cursor.execute(create_table13_query)

    create_table14_query = '''
    CREATE TABLE IF NOT EXISTS "Telemetry" (
        "TimeSeconds" INT,
        "X" DECIMAL(9, 2),
        "Y" DECIMAL(9, 2),
        "RPM" INT,
        "Speed" INT,
        "nGear" INT,
        "Throttle" INT,
        "Brake" TEXT,
        "Sector" TEXT,
        "Event" INT,
        "DriverNumber" INT
    );
    '''

    cursor.execute(create_table14_query)

    create_table15_query = '''
    CREATE TABLE IF NOT EXISTS "Track_Metadata" (
        "Round" INT,
        "Grand Prix" TEXT,
        "Circuit" TEXT,
        "Race date" TEXT,
        "Location" TEXT,
        "Type" TEXT,
        "Direction" TEXT,
        "Country" TEXT,
        "Length" INT,
        "Turns" INT,
        "Url" TEXT,
        "Total Laps" INT,
        "Fact1" TEXT,
        "Fact2" TEXT,
        "Fact3" TEXT,
        "Fact4" TEXT,
        "Fact5" TEXT
    );
    '''

    cursor.execute(create_table15_query)

    connection.commit()

    cursor.close()

    connection.close()

    return

In [ ]:
#Create_Tables_Func()

In [ ]:
#Function to delete tables from Amazon RDS database (if necessary)

def drop_tables_func(table_to_delete):

    connection = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=dbname
    )

    cursor = connection.cursor()

    drop_table_query = f"""DROP TABLE IF EXISTS "{table_to_delete}" CASCADE;"""

    cursor.execute(drop_table_query)

    connection.commit()

    cursor.close()

    connection.close()

    return

In [ ]:
#drop_tables_func('Race_Results')

#drop_tables_func('Quali_Results')

#drop_tables_func('Events')

#drop_tables_func('Race_Weather')

#drop_tables_func('Quali_Weather')

#drop_tables_func('Race_Laps')

#drop_tables_func('Telemetry_Best_Lap')

#drop_tables_func('Photo_Link')

#drop_tables_func('Logo_Link')

#drop_tables_func('Champs')

#drop_tables_func('Pred')

#drop_tables_func('Sprint_Results')

#drop_tables_func('Driver_Info')

#drop_tables_func('Telemetry')

#drop_tables_func('Track_Metadata')

In [ ]:
#Function to load information about race results.

def results_race_load(Year_input, Event_input):

  session= f1.get_session(Year_input, Event_input, 'R')

  session.load()

  results= session.results

  results["Year"]= Year_input

  results["Event"]= Event_input

  results["Session"]= "Race"

  results_race_1= results[["Year", "Event", "Session", "TeamName", "DriverNumber", "Abbreviation", "FullName", "Position", "GridPosition", "Status", "Time", "Points"]]

  results_race_1["Time_sec"]= results_race_1['Time'] / np.timedelta64(1, 's')

  results_race_1["Time"]= results_race_1["Time"].fillna(0)

  results_race_1["Time"]= results_race_1["Time"].astype(str)

  results_race= results_race_1[["Year", "Event", "Session", "TeamName", "DriverNumber", "Abbreviation", "FullName", "Position", "GridPosition", "Status", "Time", "Time_sec", "Points"]]

  return results_race

In [ ]:
#Function to load information about sprint results

def results_sprint_load(Year_input, Event_input):

    data= {"Year": [],
          "Event": [],
          "Session": [],
          "TeamName": [],
          "DriverNumber": [],
          "Abbreviation": [],
          "FullName": [],
          "Position": [],
          "GridPosition": [],
          "Status": [],
          "Time": [],
          "Points": []}

    results_sprint= pd.DataFrame(data)

    try:

        session= f1.get_session(Year_input, Event_input, 'S')

        session.load()

        results= session.results

        results["Year"]= Year_input

        results["Event"]= Event_input

        results["Session"]= "Sprint"

        results_sprint_1= results[["Year", "Event", "Session", "TeamName", "DriverNumber", "Abbreviation", "FullName", "Position", "GridPosition", "Status", "Time", "Points"]]

        results_sprint_1["Time_sec"]= results_sprint_1['Time'] / np.timedelta64(1, 's')

        results_sprint_1["Time"]= results_sprint_1["Time"].fillna(0)

        results_sprint_1["Time"]= results_sprint_1["Time"].astype(str)

        results_sprint_temp= results_sprint_1[["Year", "Event", "Session", "TeamName", "DriverNumber", "Abbreviation", "FullName", "Position", "GridPosition", "Status", "Time", "Time_sec", "Points"]]

        results_sprint= pd.concat([results_sprint, results_sprint_temp], axis= 0)

    except:

        pass

    return results_sprint

In [ ]:
#Function to load information about quali results.

def results_quali_load(Year_input, Event_input):

  session= f1.get_session(Year_input, Event_input, 'Q')

  session.load()

  results= session.results

  results["Year"]= Year_input

  results["Event"]= Event_input

  results["Session"]= "Quali"

  results_1= results[["Year", "Event", "Session", "TeamName", "DriverNumber", "Abbreviation", "FullName", "Position", "Q1", "Q2", "Q3"]]

  results_quali_1= results_1.rename(columns={'Position': 'ClassifiedPosition'})

  results_quali_1["Q1_sec"]= results_quali_1['Q1'] / np.timedelta64(1, 's')

  results_quali_1["Q2_sec"]= results_quali_1['Q2'] / np.timedelta64(1, 's')

  results_quali_1["Q3_sec"]= results_quali_1['Q3'] / np.timedelta64(1, 's')

  results_quali_1["Q1"]= results_quali_1["Q1"].fillna(0)

  results_quali_1["Q1"]= results_quali_1["Q1"].astype(str)

  results_quali_1["Q2"]= results_quali_1["Q2"].fillna(0)

  results_quali_1["Q2"]= results_quali_1["Q2"].astype(str)

  results_quali_1["Q3"]= results_quali_1["Q3"].fillna(0)

  results_quali_1["Q3"]= results_quali_1["Q3"].astype(str)

  results_quali= results_quali_1[["Year", "Event", "Session", "TeamName", "DriverNumber", "Abbreviation", "FullName", "ClassifiedPosition", "Q1", "Q1_sec", "Q2", "Q2_sec", "Q3", "Q3_sec"]]

  return results_quali

In [ ]:
#Function to load information about the event schedule.

def event_func_load(Year_input, Event_input):

    event_0= f1.get_event(Year_input, Event_input)

    event_1= pd.DataFrame(event_0)

    event_2= event_1.reset_index()

    event_3= event_2.set_index('index').T

    event_4= event_3[["RoundNumber", "Location", "OfficialEventName", "EventDate", "EventName", "EventFormat", "Session4Date", "Session4DateUtc", "Session5Date", "Session5DateUtc"]]

    event_5= event_4.rename(columns={"Session4Date": "Quali_Date", "Session4DateUtc": "Quali_DateUtc", "Session5Date": "Race_Date", "Session5DateUtc": "Race_DateUtc"})

    event_5["Year"]= int(Year_input)

    event_5["Event"]= int(Event_input)

    event_5["RoundNumber"]= event_5["RoundNumber"].astype(int)

    event= event_5[["Year", "Event", "RoundNumber", "Location", "OfficialEventName", "EventDate", "EventName", "EventFormat", "Quali_Date", "Quali_DateUtc", "Race_Date", "Race_DateUtc"]]

    return event

In [ ]:
#Function to load information about the weather on race day.

def race_weather_func_load(Year_input, Event_input):

    session = f1.get_session(Year_input, Event_input, "R")

    session.load()

    weather_0= session.weather_data

    weather_0["Year"]= Year_input

    weather_0["Event"]= Event_input

    weather_0["Session"]= "Race"

    weather_0["Time_sec"]= weather_0['Time'] / np.timedelta64(1, 's')

    weather_0["Time"]= weather_0["Time"].fillna(0)

    weather_0["Time"]= weather_0["Time"].astype(str)

    weather_race= weather_0[["Year", "Event", "Session", "Time", "Time_sec", "AirTemp", "Humidity", "Pressure", "Rainfall", "TrackTemp", "WindDirection", "WindSpeed"]]

    return weather_race

In [ ]:
#Function to load information about the weather on quali day.

def quali_weather_func_load(Year_input, Event_input):

    session = f1.get_session(Year_input, Event_input, "Q")

    session.load()

    weather_0= session.weather_data

    weather_0["Year"]= Year_input

    weather_0["Event"]= Event_input

    weather_0["Session"]= "Quali"

    weather_0["Time_sec"]= weather_0['Time'] / np.timedelta64(1, 's')

    weather_0["Time"]= weather_0["Time"].fillna(0)

    weather_0["Time"]= weather_0["Time"].astype(str)

    weather_quali= weather_0[["Year", "Event", "Session", "Time", "Time_sec", "AirTemp", "Humidity", "Pressure", "Rainfall", "TrackTemp", "WindDirection", "WindSpeed"]]

    return weather_quali

In [ ]:
#Function to load information for each lap of each driver in the race.

def laps_race_load(Year_input, Event_input):

    session= f1.get_session(Year_input, Event_input, 'R')

    session.load()

    laps_1= session.laps

    laps_2= laps_1[["Time", "Driver", "DriverNumber", "LapTime", "LapNumber", "PitOutTime", "PitInTime", "Sector1Time", "Sector1SessionTime", "Sector2Time", "Sector2SessionTime",
                    "Sector3Time", "Sector3SessionTime", "SpeedI1", "SpeedI2", "SpeedFL", "SpeedST", "Compound", "TyreLife", "FreshTyre", "Team", "LapStartTime", "LapStartDate",
                    "TrackStatus", "Position"]]

    laps_2["Year"]= Year_input

    laps_2["Event"]= Event_input

    laps_2["Session"]= "Race"

    laps_2["Time_sec"]= laps_2['Time'] / np.timedelta64(1, 's')

    laps_2["LapTime_sec"]= laps_2['LapTime'] / np.timedelta64(1, 's')

    laps_2["PitOutTime_sec"]= laps_2['PitOutTime'] / np.timedelta64(1, 's')

    laps_2["PitInTime_sec"]= laps_2['PitInTime'] / np.timedelta64(1, 's')

    laps_2["Sector1Time_sec"]= laps_2['Sector1Time'] / np.timedelta64(1, 's')

    laps_2["Sector2Time_sec"]= laps_2['Sector2Time'] / np.timedelta64(1, 's')

    laps_2["Sector3Time_sec"]= laps_2['Sector3Time'] / np.timedelta64(1, 's')

    laps_2["Sector1SessionTime_sec"]= laps_2['Sector1SessionTime'] / np.timedelta64(1, 's')

    laps_2["Sector2SessionTime_sec"]= laps_2['Sector2SessionTime'] / np.timedelta64(1, 's')

    laps_2["Sector3SessionTime_sec"]= laps_2['Sector3SessionTime'] / np.timedelta64(1, 's')

    laps_2["LapStartTime_sec"]= laps_2['LapStartTime'] / np.timedelta64(1, 's')

    laps_2["Time"]= laps_2["Time"].fillna(0)

    laps_2["Time"]= laps_2["Time"].astype(str)

    laps_2["LapTime"]= laps_2["LapTime"].fillna(0)

    laps_2["LapTime"]= laps_2["LapTime"].astype(str)

    laps_2["PitOutTime"]= laps_2["PitOutTime"].fillna(0)

    laps_2["PitOutTime"]= laps_2["PitOutTime"].astype(str)

    laps_2["PitInTime"]= laps_2["PitInTime"].fillna(0)

    laps_2["PitInTime"]= laps_2["PitInTime"].astype(str)

    laps_2["Sector1Time"]= laps_2["Sector1Time"].fillna(0)

    laps_2["Sector1Time"]= laps_2["Sector1Time"].astype(str)

    laps_2["Sector2Time"]= laps_2["Sector2Time"].fillna(0)

    laps_2["Sector2Time"]= laps_2["Sector2Time"].astype(str)

    laps_2["Sector3Time"]= laps_2["Sector3Time"].fillna(0)

    laps_2["Sector3Time"]= laps_2["Sector3Time"].astype(str)

    laps_2["Sector1SessionTime"]= laps_2["Sector1SessionTime"].fillna(0)

    laps_2["Sector1SessionTime"]= laps_2["Sector1SessionTime"].astype(str)

    laps_2["Sector2SessionTime"]= laps_2["Sector2SessionTime"].fillna(0)

    laps_2["Sector2SessionTime"]= laps_2["Sector2SessionTime"].astype(str)

    laps_2["Sector3SessionTime"]= laps_2["Sector3SessionTime"].fillna(0)

    laps_2["Sector3SessionTime"]= laps_2["Sector3SessionTime"].astype(str)

    laps_2["LapStartTime"]= laps_2["LapStartTime"].fillna(0)

    laps_2["LapStartTime"]= laps_2["LapStartTime"].astype(str)

    laps= laps_2[["Year", "Event", "Session", "Time", "Time_sec", "Driver", "DriverNumber", "LapTime", "LapTime_sec", "LapNumber", "PitOutTime", "PitOutTime_sec", "PitInTime", "PitInTime_sec",
                  "Sector1Time", "Sector1Time_sec", "Sector2Time", "Sector2Time_sec", "Sector3Time", "Sector3Time_sec", "Sector1SessionTime", "Sector1SessionTime_sec", "Sector2SessionTime",
                  "Sector2SessionTime_sec", "Sector3SessionTime", "Sector3SessionTime_sec", "SpeedI1", "SpeedI2", "SpeedFL", "SpeedST", "Compound", "TyreLife", "FreshTyre", "Team",
                  "LapStartTime", "LapStartTime_sec", "LapStartDate", "TrackStatus", "Position"]]

    return laps

In [ ]:
#Function to load telemetry information of the fastest lap of each driver in the race

def telemetry_race_load(Year_input, Event_input):

    session = f1.get_session(Year_input, Event_input, "R")

    session.load()

    data= {"Year": [],
          "Event": [],
          "Session": [],
          "DriverNumber": [],
          "X": [],
          "Y": [],
          "Z": [],
          "Speed": [],
          "nGear": [],
          "RPM": [],
          "Throttle": [],
          "Brake": [],
          "Date": [],
          "Time": [],
          "Time_sec": [],
          "SessionTime": [],
          "SessionTime_sec": []}

    telemetry_bl= pd.DataFrame(data)

    for driver in session.drivers:

        try:

            lap = session.laps.pick_driver(driver).pick_fastest()

            telemetry_1= lap.telemetry[["X", "Y", "Z", "Speed", "nGear", "RPM", "Throttle", "Brake", "Date", "Time", "SessionTime"]]

            telemetry_1["Year"]= Year_input

            telemetry_1["Event"]= Event_input

            telemetry_1["Session"]= "Race"

            telemetry_1["DriverNumber"]= int(driver)

            telemetry_1["Time_sec"]= telemetry_1['Time'] / np.timedelta64(1, 's')

            telemetry_1["Time"]= telemetry_1["Time"].fillna(0)

            telemetry_1["Time"]= telemetry_1["Time"].astype(str)

            telemetry_1["SessionTime_sec"]= telemetry_1['SessionTime'] / np.timedelta64(1, 's')

            telemetry_1["SessionTime"]= telemetry_1["SessionTime"].fillna(0)

            telemetry_1["SessionTime"]= telemetry_1["SessionTime"].astype(str)

            telemetry_bl= pd.concat([telemetry_bl, telemetry_1], axis= 0)

        except:

            pass

    return telemetry_bl

In [ ]:
#Function to append the uploaded information of the results of the race to the Amazon RDS database

def append_race_results_func(Year_input, Event_input):

    connection = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=dbname
    )

    cursor = connection.cursor()

    delete_query = '''DELETE FROM "Race_Results" WHERE "Year" = %s AND "Event" = %s;'''

    cursor.execute(delete_query, (Year_input, Event_input))

    connection.commit()

    cursor.close()

    connection.close()

    engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))

    results_race.to_sql('Race_Results', engine, if_exists='append', index=False)

    return

In [ ]:
#Function to append the uploaded information of the results of the quali to the Amazon RDS database

def append_quali_results_func(Year_input, Event_input):

    connection = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=dbname
    )

    cursor = connection.cursor()

    delete_query = '''DELETE FROM "Quali_Results" WHERE "Year" = %s AND "Event" = %s;'''

    cursor.execute(delete_query, (Year_input, Event_input))

    connection.commit()

    cursor.close()

    connection.close()

    engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))

    results_quali.to_sql('Quali_Results', engine, if_exists='append', index=False)

    return

In [ ]:
#Function to append the uploaded information of the event schedule to the Amazon RDS database

def append_events_func(Year_input, Event_input):

    connection = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=dbname
    )

    cursor = connection.cursor()

    delete_query = '''DELETE FROM "Events" WHERE "Year" = %s AND "Event" = %s;'''

    cursor.execute(delete_query, (Year_input, Event_input))

    connection.commit()

    cursor.close()

    connection.close()

    engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))

    event.to_sql('Events', engine, if_exists='append', index=False)

    return

In [ ]:
#Function to append the uploaded information of the weather of each race to the Amazon RDS database

def append_race_weather_func(Year_input, Event_input):

    connection = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=dbname
    )

    cursor = connection.cursor()

    delete_query = '''DELETE FROM "Race_Weather" WHERE "Year" = %s AND "Event" = %s;'''

    cursor.execute(delete_query, (Year_input, Event_input))

    connection.commit()

    cursor.close()

    connection.close()

    engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))

    weather_race.to_sql('Race_Weather', engine, if_exists='append', index=False)

    return

In [ ]:
#Function to append the uploaded information of the weather of each quali to the Amazon RDS database

def append_quali_weather_func(Year_input, Event_input):

    connection = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=dbname
    )

    cursor = connection.cursor()

    delete_query = '''DELETE FROM "Quali_Weather" WHERE "Year" = %s AND "Event" = %s;'''

    cursor.execute(delete_query, (Year_input, Event_input))

    connection.commit()

    cursor.close()

    connection.close()

    engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))

    weather_quali.to_sql('Quali_Weather', engine, if_exists='append', index=False)

    return

In [ ]:
#Function to append the uploaded information of every lap of each driver of the race to the Amazon RDS database

def append_race_laps_func(Year_input, Event_input):

    connection = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=dbname
    )

    cursor = connection.cursor()

    delete_query = '''DELETE FROM "Race_Laps" WHERE "Year" = %s AND "Event" = %s;'''

    cursor.execute(delete_query, (Year_input, Event_input))

    connection.commit()

    cursor.close()

    connection.close()

    engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))

    laps.to_sql('Race_Laps', engine, if_exists='append', index=False)

    return

In [ ]:
#Function to append the uploaded telemetry information of the best lap of each driver in the race to the Amazon RDS database

def append_telemetry_race_func(Year_input, Event_input):

    connection = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=dbname
    )

    cursor = connection.cursor()

    delete_query = '''DELETE FROM "Telemetry_Best_Lap" WHERE "Year" = %s AND "Event" = %s;'''

    cursor.execute(delete_query, (Year_input, Event_input))

    connection.commit()

    cursor.close()

    connection.close()

    engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))

    telemetry_bl.to_sql('Telemetry_Best_Lap', engine, if_exists='append', index=False)

    return

In [ ]:
#Function to append the uploaded information of the results of the race to the Amazon RDS database

def append_sprint_results_func(Year_input, Event_input):

    connection = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=dbname
    )

    cursor = connection.cursor()

    delete_query = '''DELETE FROM "Sprint_Results" WHERE "Year" = %s AND "Event" = %s;'''

    cursor.execute(delete_query, (Year_input, Event_input))

    connection.commit()

    cursor.close()

    connection.close()

    engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))

    results_sprint.to_sql('Sprint_Results', engine, if_exists='append', index=False)

    return

In [ ]:
#Cell to execute previous functions

Year_input= 2024

#Event_input= 21

for Event_input in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]:

#results_race= results_race_load(Year_input, Event_input)

#append_race_results_func(Year_input, Event_input)

#results_quali= results_quali_load(Year_input, Event_input)

#append_quali_results_func(Year_input, Event_input)

#event= event_func_load(Year_input, Event_input)

#append_events_func(Year_input, Event_input)

#weather_race= race_weather_func_load(Year_input, Event_input)

#append_race_weather_func(Year_input, Event_input)

#weather_quali= quali_weather_func_load(Year_input, Event_input)

#append_quali_weather_func(Year_input, Event_input)

#laps= laps_race_load(Year_input, Event_input)

#append_race_laps_func(Year_input, Event_input)

#    telemetry_bl= telemetry_race_load(Year_input, Event_input)

#    append_telemetry_race_func(Year_input, Event_input)

#    print(Event_input)

#    results_sprint= results_sprint_load(Year_input, Event_input)

#    append_sprint_results_func(Year_input, Event_input)

core           INFO 	Loading data for Chinese Grand Prix - Sprint [v3.4.4]
INFO:fastf1.fastf1.core:Loading data for Chinese Grand Prix - Sprint [v3.4.4]
req            INFO 	Using cached data for session_info
INFO:fastf1.fastf1.req:Using cached data for session_info
req            INFO 	Using cached data for driver_info
INFO:fastf1.fastf1.req:Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
INFO:fastf1.fastf1.req:Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
INFO:fastf1.fastf1.req:Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
INFO:fastf1.fastf1.req:Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
INFO:fastf1.fastf1.req:Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
INFO:fastf1.fastf1.req:Using cached data for timing_app_data
core     

In [ ]:
#Generate new user with SELECT permissions

def New_user_func(new_user, new_password):

    connection = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=dbname
    )

    cursor = connection.cursor()

    #Create the new user
    try:
        cursor.execute(f"CREATE USER {new_user} WITH PASSWORD '{new_password}';")
        print(f"User '{new_user}' successfully created.")
    except psycopg2.errors.DuplicateObject:
        print(f"The user '{new_user}' already exists.")

    #Grant read only premissions
    try:
        cursor.execute(f"GRANT CONNECT ON DATABASE postgres TO {new_user};")
        cursor.execute(f"GRANT USAGE ON SCHEMA public TO {new_user};")
        cursor.execute(f"GRANT SELECT ON ALL TABLES IN SCHEMA public TO {new_user};")
        cursor.execute(f"ALTER DEFAULT PRIVILEGES IN SCHEMA public GRANT SELECT ON TABLES TO {new_user};")
        print(f"Read only permissions granted to '{new_user}'.")
    except Exception as e:
        print("Error while granting permissions:", e)

    # Confirm changes and close connection
    connection.commit()
    cursor.close()
    connection.close()

    return

In [ ]:
new_user = ""

new_password = ""

New_user_func(new_user, new_password)

User 'fwin_readonly2' successfully created.
Read only permissions granted to 'fwin_readonly2'.


In [ ]:
xlsx= pd.ExcelFile("{}/Photos.xlsx".format(Ruta))

Drivers= pd.read_excel(xlsx, "Drivers")

Teams= pd.read_excel(xlsx, "Teams")

Champs= pd.read_excel(xlsx, "Champs")

In [ ]:
xlsx= pd.ExcelFile("{}/pred_2024.xlsx".format(Ruta))

Pred_1= pd.read_excel(xlsx)

Pred= Pred_1.rename(columns={"year": "Year", "Raw Predicted Position": "RawPredictedPosition", "Clean Predicted Position": "CleanPredictedPosition", "Actual Position": "ActualPosition"})

In [ ]:
driver_info= pd.read_csv("{}/driver_info.csv".format(Ruta))

telemetry= pd.read_csv("{}/telemetry.csv".format(Ruta))

In [ ]:
track_metadata= pd.read_csv("{}/Track_Metadata.csv".format(Ruta))

In [ ]:
#Function to append Drivers table

def append_photo_link_func():

    connection = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=dbname
    )

    cursor = connection.cursor()

    delete_query = '''DELETE FROM "Photo_Link";'''

    cursor.execute(delete_query)

    connection.commit()

    cursor.close()

    connection.close()

    engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))

    Drivers.to_sql('Photo_Link', engine, if_exists='append', index=False)

    return

In [ ]:
#Function to append Teams table

def append_logo_link_func():

    connection = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=dbname
    )

    cursor = connection.cursor()

    delete_query = '''DELETE FROM "Logo_Link";'''

    cursor.execute(delete_query)

    connection.commit()

    cursor.close()

    connection.close()

    engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))

    Teams.to_sql('Logo_Link', engine, if_exists='append', index=False)

    return

In [ ]:
#Function to append Champs table

def append_champ_func():

    connection = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=dbname
    )

    cursor = connection.cursor()

    delete_query = '''DELETE FROM "Champs";'''

    cursor.execute(delete_query)

    connection.commit()

    cursor.close()

    connection.close()

    engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))

    Champs.to_sql('Champs', engine, if_exists='append', index=False)

    return

In [ ]:
#Function to append Pred table

def append_pred_func():

    connection = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=dbname
    )

    cursor = connection.cursor()

    delete_query = '''DELETE FROM "Pred";'''

    cursor.execute(delete_query)

    connection.commit()

    cursor.close()

    connection.close()

    engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))

    Pred.to_sql('Pred', engine, if_exists='append', index=False)

    return

In [ ]:
#Function to append Driver_Info table

def append_diver_info_func():

    connection = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=dbname
    )

    cursor = connection.cursor()

    delete_query = '''DELETE FROM "Driver_Info";'''

    cursor.execute(delete_query)

    connection.commit()

    cursor.close()

    connection.close()

    engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))

    driver_info.to_sql('Driver_Info', engine, if_exists='append', index=False)

    return

In [ ]:
#Function to append telemetry table

def append_telemetry_func():

    connection = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=dbname
    )

    cursor = connection.cursor()

    delete_query = '''DELETE FROM "Telemetry";'''

    cursor.execute(delete_query)

    connection.commit()

    cursor.close()

    connection.close()

    engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))

    telemetry.to_sql('Telemetry', engine, if_exists='append', index=False)

    return

In [ ]:
#Function to append Track_Metadata table

def append_track_metadata_func():

    connection = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=dbname
    )

    cursor = connection.cursor()

    delete_query = '''DELETE FROM "Track_Metadata";'''

    cursor.execute(delete_query)

    connection.commit()

    cursor.close()

    connection.close()

    engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))

    track_metadata.to_sql('Track_Metadata', engine, if_exists='append', index=False)

    return

In [ ]:
#append_photo_link_func()

#append_logo_link_func()

#append_champ_func()

#append_pred_func()

#append_diver_info_func()

#append_telemetry_func()

#append_track_metadata_func()